## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tateyama,JP,34.98,139.87,70.00,88,75,13.87,broken clouds
1,1,Ushuaia,AR,-54.80,-68.30,46.40,75,75,23.04,broken clouds
2,2,Cape Town,ZA,-33.93,18.42,64.99,82,75,16.11,broken clouds
3,3,Bethel,US,41.37,-73.41,53.01,100,1,4.74,clear sky
4,4,Gizo,SB,-8.10,156.84,84.65,67,63,1.10,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_temp_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                (city_data_df["Max Temp"] >= min_temp)]
city_temp_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tateyama,JP,34.98,139.87,70.00,88,75,13.87,broken clouds
2,2,Cape Town,ZA,-33.93,18.42,64.99,82,75,16.11,broken clouds
28,28,Jiaozhou,CN,36.28,120.00,68.00,25,0,11.18,clear sky
32,32,Fukue,JP,32.69,128.84,68.00,72,75,6.93,broken clouds
36,36,Muisne,EC,0.60,-80.03,74.30,81,98,3.76,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
city_temp_df.count()

Unnamed: 0             182
City                   182
Country                178
Lat                    182
Lng                    182
Max Temp               182
Humidity               182
Cloudiness             182
Wind Speed             182
Current Description    182
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_temp_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tateyama,JP,70.00,broken clouds,34.98,139.87,
2,Cape Town,ZA,64.99,broken clouds,-33.93,18.42,
28,Jiaozhou,CN,68.00,clear sky,36.28,120.00,
32,Fukue,JP,68.00,broken clouds,32.69,128.84,
36,Muisne,EC,74.30,overcast clouds,0.60,-80.03,
37,Hermanus,ZA,66.99,clear sky,-34.42,19.23,
38,Cabo San Lucas,MX,73.99,clear sky,22.89,-109.91,
41,Rikitea,PF,71.55,broken clouds,-23.12,-134.97,
43,Pisco,PE,64.40,overcast clouds,-13.70,-76.22,
46,Clarence Town,AU,66.00,shower rain,-32.58,151.78,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key    
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel " + hotels["results"][0]["name"] + "was the closest hotel found.")
    except:
        print("No hotel found skipping.")
        pass
hotel_df.head()     

No hotel found skipping.
No hotel found skipping.
No hotel found skipping.
No hotel found skipping.
No hotel found skipping.
No hotel found skipping.
No hotel found skipping.
No hotel found skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tateyama,JP,70.00,broken clouds,34.98,139.87,Tateyama Seaside Hotel
2,Cape Town,ZA,64.99,broken clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
28,Jiaozhou,CN,68.00,clear sky,36.28,120.00,Sheraton Qingdao Jiaozhou Hotel
32,Fukue,JP,68.00,broken clouds,32.69,128.84,Goto Guesthouse Utojuku & SA Car Rental
36,Muisne,EC,74.30,overcast clouds,0.60,-80.03,Quinta Samay


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [46]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [47]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))